# Decorators

## Higher order functions

Functions can be passed as arguments to another function.

Such functions that take other functions as arguments are also called **higher order functions** (ex: `property`, `staticmethod`, `atexit.register`, ...)

## Nested functions

A function can define and return another function.

Here, `is_returned()` is a **nested (or inner) function** which is defined and returned each time we call `is_called()`.

In [33]:
def is_called():
    def is_returned():
        print("Hello")
    return is_returned

result = is_called() # result is an alias of is_returned
result()

Hello


## Python closures

A **closure** is a nested function which has access to a **free variable** from an enclosing function that has finished its execution. 

The three characteristics of a Python closure are:

1. it is a nested function
2. it has access to a free variable in outer scope
3. it is returned from the enclosing function

A **free variable** is a variable that is not bound in the local scope. 

The following `print_message` function returns a `printer` function which is assigned to the `myprinter` variable. 

At this moment, `print_message` has finished its execution. However, the `printer` closure still has access to the `msg` and `name` variables.

In [35]:
def print_message(name):

    msg = "Hello"

    def printer():
        print(f"{msg} {name}!")

    return printer


myprinter = print_message("Vadim")
myprinter()

hello Vadim!


In order for closures to rebind (assign) a free variables, it has to be prefixed by the **nonlocal** keyword.

In [40]:
def make_counter():
    count = 0
    def inner():
        nonlocal count
        count += 1
        return count
    return inner


inc_counter = make_counter()
c = inc_counter()
print(c)
c = inc_counter()
print(c)
c = inc_counter()
print(c)

1
2
3


## Decorators

**Decorators** are Python objects that can register, annotate, and/or wrap a Python function, method or class. 

A decorator is a callable object that accepts one argument the function (the method or the class) being decorated.

The return value of the decorator replaces the original function (method or class) definition.

Decorators may appear before any function, inner function, method or class definition.

For example, the builtin function `staticmethod()` can be used to transform an instance method into a static method.

Without the decorator feature, a program that uses this function looks something like:

In [9]:
class Demo:
    def sayHello():
        print("Hello")
    sayHello=staticmethod(sayHello)
    
Demo.sayHello()

Hello


The decorator version looks like this:

In [10]:
class Demo:
    @staticmethod
    def sayHello():
        print("Hello")
    
Demo.sayHello()

Hello


What are the advantages of the decorator syntax?

1. You are made aware that the method is a static method before you read the method body, giving you a better context for understanding the method.
2. The method name is not repeated.

Decorator are often used to annotate a function (by adding attributes to it), or wrapping a function with another function, then returning the wrapper (which replaces the original function).

**Note**: a decorator may return any kind of objects, which means that for advanced uses, you can turn functions or methods into specialized objects of your own choosing.

## Stacking decorators

You can stack multiple decorators on the same function (method or class) definition, one per line.

The ordering of the decorators determines the structure of the result.

When you write:

In [11]:
@dec2
@dec1
def func(arg1, arg2):
    pass

SyntaxError: invalid syntax (<ipython-input-11-05b1729054f9>, line 3)

This is equivalent to:

In [12]:
def func(arg1, arg2):
    pass
func = dec2(dec1(func))

NameError: name 'dec2' is not defined

## Classes as decorators

Functions and methods are called **callable** as they can be called.

In fact, any object which implements the special **`__call__()`** method is a callable object. 

A decorator can be defined as a *callable* class.

Recall that the decorator syntax:

`@my_decorator
 def func:
    pass`
    
is just an easier way of saying:

`func = my_decorator(func)` 

Therefore, if `my_decorator` is a class, it needs to take func as an argument in its `__init__()` method. Furthermore, the class instance needs to be callable so that it can stand in for the decorated function.

The following class, for instance, can be used as a decorator:

In [3]:
class traced:
    def __init__(self,func):
        self.func = func
    def __call__(self,*__args,**__kw):
        print("entering", self.func)
        try:
            return self.func(*__args,**__kw)
        finally:
            print("exiting", self.func)


After having decorated the following `hello()` function with `@traced` `hello` is no longer a function, but is instead an instance of the `traced` class that has the old `hello` function saved in its `func` attribute.

In [6]:
@traced
def hello(name):
    print(f"Hello, my name is {name}!")
    
hello("Marco")

entering <function hello at 0x0000000005782BF8>
Hello, my name is Marco!
exiting <function hello at 0x0000000005782BF8>


## Functions as Decorators

Most decorators expect an actual function as their input and return a function. 

To make our previous decorator (`traced`) which return a class instance be compatible with a wider range of decorators, we can modify it to return a function.

The following decorator provides the same functionality as the original `traced` decorator, but instead of returning a `traced` object instance, it returns a new function object that wraps the original function.

In [13]:
def traced(func):
    def wrapper(*__args,**__kw):
        print("entering", func)
        try:
            return func(*__args,**__kw)
        finally:
            print("exiting", func)
    return wrapper

@traced
def hello(name):
    print(f"Hello, my name is {name}!")
    
hello("Marco")

entering <function hello at 0x00000000057A40D0>
Hello, my name is Marco!
exiting <function hello at 0x00000000057A40D0>


## Inner functions and closures

When you define a function inside of another function, any undefined local variables in the inner function will take the value of that variable in the outer function.

In the preceding decorator, the value of `func` in the inner function comes from the value of `func` in the outer function.

Because the inner function definition is executed each time the outer function is called, Python actually creates a new wrapper function object each time. Such function objects are called "**lexical closures**" because they enclose a set of variables from the lexical scope where the function was defined.

A closure does not actually duplicate the code of the function, however; it simply encloses a reference to the existing code, and a reference to the free variables from the enclosing function. 
In this case, that means that the wrapper closure is essentially a pointer to the Python bytecode making up the wrapper function body, and a pointer to the local variables of the `traced` function during the invocation when the closure was created.

Because a closure is really just a normal Python function object (with some predefined variables), and because most decorators expect to receive a function object, creating a closure is the most popular way of creating a ***stackable decorator***.

## Decorators with Arguments

It is possible to pass one or more arguments to a decorator.

When you write:

`@decomaker(argA, argB, ...)
 def func(arg1, arg2, ...):
    pass`

This is equivalent to:

`func = decomaker(argA, argB, ...)(func)`

For instance you may decide to create a `@require` decorator that will give the possibility to record a method's precondition.

Here is a simplified version (inheritance is not taken into account for instance) of what this decorator may look like:

In [3]:
def require(expr):
    def decorator(func):
        def wrapper(*args,**kw):
            assert eval(expr),f"Precond {expr} failed"
            return func(*args,**kw)
        return wrapper
    return decorator

@require("len(args)==1")
def test(*args):
    print(args[0])
    
test("Hello world!")

test(12,23) # Exception raised  (Assertion Error)!

Hello world!


AssertionError: Precond len(args)==1 failed

The `require` decorator creates two closures. 

The first closure creates a decorator function that knows the `expr` that was supplied to `@require`. This means `require` itself is not really the decorator function here. Instead, `require` returns the decorator function, here called`decorator`. 

This is very different from the previous decorators, and this change is necessary to implement parameterized decorators.

The second closure is the actual wrapper function that evaluates `expr` whenever the original function is called.

**Note**: decorator invocations follow the same syntax rules as normal Python function or method calls, so you can use positional arguments, keyword arguments, or both.

## Safe decorators

It’s easy to create a decorator that will work by itself, but creating a decorator that will work properly when combined with other decorators is a bit more complex.

To the extent possible, a decorator should return an actual function object, with the same name and attributes as the original function, so as not to confuse an outer decorator or cancel out the work of an inner decorator.

This means that decorators that simply modify and return the function they were given (like `@author`), are already safe. But decorators that return a wrapper function need to do two more things to be safe:
1. Set the new function's name to match the old function's name.
2. Copy the old function's attributes to the new function.

The `@require` decorator has been modified to follow these recommendations:

In [21]:
def require(expr):
    def decorator(func):
        def wrapper(*__args,**__kw):
            assert eval(expr),f"Precond {expr} failed"
            return func(*__args,**__kw)
        wrapper.__name__ = func.__name__
        wrapper.__dict__ = func.__dict__
        wrapper.__doc__ = func.__doc__
        return wrapper
    return decorator

The **functools.wraps()** function is a facility that does exactly the same.

`wraps()` takes a function used in a decorator and adds the functionality of copying over the function name, docstring, arguments list, etc.

And since `wraps()` is itself a decorator, the following code does the correct thing:

In [22]:
def require(expr):
    def decorator(func):
        @wraps(func)
        def wrapper(*__args,**__kw):
            assert eval(expr),"Precond {expr} failed"
            return func(*__args,**__kw)
        return wrapper
    return decorator

## Stateful Decorators

Sometimes, it’s useful to have a decorator that can keep track of state.

We can use classes to keep state, but, in simple cases, you can also get away with using function attributes.

Function attributes let you record arbitrary values as attributes on a function object.

For example, suppose you want to track the author of a function or method, using an `@author` decorator. In the following example, you simply set an `author_name` attribute on a function and return it, rather than creating a wrapper. 

Then, you can retrieve the attribute at a later time as part of some metadata-gathering operation.

In [16]:
def author(author_name):
    def decorator(func):
        func.author_name = author_name
        return func
    return decorator

@author("John Martin")
def sequenceOf(param1, param2):
    pass

print(sequenceOf.author_name)

John Martin


The typical way to maintain state is by using classes.

As a simple example, here we create a decorator that counts the number of times a function is called.

The `__init__()` method must store a reference to the function and can do any other necessary initialization. The `__call__()` method will be called instead of the decorated function. 

It does essentially the same thing as the `wrapper()` function in our earlier examples. 

Note that you need to use the `functools.update_wrapper()` function instead of `@functools.wraps`.

In [3]:
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def say_hello():
    print("Hello!")
    
say_hello()
say_hello()
say_hello.num_calls

Call 1 of 'say_hello'
Hello!
Call 2 of 'say_hello'
Hello!


2